<a href="https://colab.research.google.com/github/kangning-huang/PopWeightedUHI/blob/main/01_pop_weighted_UHI_timeseries2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Install and import

In [1]:
!pip install geemap &> /dev/null

In [2]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np

from scipy import stats

from tqdm import tqdm

import geemap
import json
import os
import requests
from geemap import geojson_to_ee, ee_to_geojson
# from ipyleaflet import GeoJSON

import altair as alt
import folium

import ee
ee.Authenticate()
ee.Initialize()

To authorize access needed by Earth Engine, open the following URL in a web browser and follow the instructions. If the web browser does not start automatically, please manually browse the URL below.

    https://code.earthengine.google.com/client-auth?scopes=https%3A//www.googleapis.com/auth/earthengine%20https%3A//www.googleapis.com/auth/devstorage.full_control&request_id=OFsoa-dVNOUUWVjxZcBAu0uecLizHtD9T4LZIP3HEOk&tc=h903Tce3z8h4i2j5jehCTBzS8gFZvfqZxITFkrBNAL0&cc=YMEJ8beCGD-z-XmVd756sA-3wlN7KKtGrGSMJDPyy7s

The authorization workflow will generate a code, which you should paste in the box below.
Enter verification code: 4/1AdQt8qiujMVEo5eLUHNpSHjS0RfLT9Ggk5AbxbapgxHQNkIY2eyqWfHqnEc

Successfully saved authorization token.


In [3]:
!pip install geopandas &> /dev/null

In [4]:
import geopandas as gdp
import pandas as pd

In [5]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


# Load Data

## Load FUAs

In [6]:
gdf_FUAs = gdp.read_file('/content/drive/MyDrive/Research_Projects/2022_UHI_vs_GHG/data/GHS_FUA_UCDB2015_GLOBE_R2019A_54009_1K_V1_0/GHS_FUA_UCDB2015_GLOBE_R2019A_54009_1K_V1_0.gpkg')
gdf_FUAs = gdf_FUAs.sort_values('FUA_p_2015', ascending=False)
# print(gdf_FUAs.head(5))

gdf_Shanghai = gdf_FUAs.loc[gdf_FUAs['eFUA_name']=='Shanghai']
gdf_Helsinki = gdf_FUAs.loc[gdf_FUAs['eFUA_name']=='Helsinki']
fc_Shanghai = geemap.geopandas_to_ee(gdf_Shanghai)
fc_Helsinki = geemap.geopandas_to_ee(gdf_Helsinki)
Map = geemap.Map()
Map.addLayer(fc_Shanghai,{},'Shanghai')
Map.center_object(fc_Shanghai.geometry())
Map

Map(center=[20, 0], controls=(WidgetControl(options=['position', 'transparent_bg'], widget=HBox(children=(Togg…

## Load Urban Land

In [13]:
def GetUrbanLandByYear(year):
  urbanYearCode = ee.Image("Tsinghua/FROM-GLC/GAIA/v10")
  urban = urbanYearCode.gt(2019-year)
  return urban.updateMask(urban)

visParms_urb = {
  'bands': ['change_year_index'],
  'min': 1,
  'max': 34,
  'palette': ['blue', 'yellow', 'red']
}

Map = geemap.Map()
# Map.addLayer(urban, visParms_urb, 'Urban change year')
Map.addLayer(GetUrbanLandByYear(2004), {'palette':['red']}, 'Urban 2004')
Map.addLayer(GetUrbanLandByYear(2010), {'palette':['red']}, 'Urban 2010')
Map.addLayer(GetUrbanLandByYear(2018), {'palette':['red']}, 'Urban 2018')
Map.center_object(fc_Shanghai)
Map

Map(center=[20, 0], controls=(WidgetControl(options=['position', 'transparent_bg'], widget=HBox(children=(Togg…

## Load UHIs

In [14]:
# Function to get UHI stats by year
def GetUhiStatsByYear(year):
  # getting the image collection
  uhiModis1 = ee.ImageCollection('users/lorenzomentaschi/uhiModis');
  uhiModis2 = ee.ImageCollection('users/lucfeyen3/uhiModis');
  uhiModis = uhiModis1.merge(uhiModis2);
  # Filter images in the year
  filterYear = ee.Filter.calendarRange(year, year, 'year')
  # Filter pixles with more than 50 rual pixels
  def FilterByMinNRuralPixel(img):
    minNRuralPixel = 50
    ruralPxlCnt = img.select('N_NOTURB_PIX_Day')
    msk = ruralPxlCnt.gte(minNRuralPixel)
    return img.select(['UHI_Day_1km', 'UHI_Night_1km']).updateMask(msk)
  # Apply both filters
  ImColYear = ee.ImageCollection(uhiModis.filter(filterYear).map(FilterByMinNRuralPixel))
  ImStatsYear = ImColYear.reduce(ee.Reducer.percentile([95, 99]))
  return ImStatsYear

# Add test layer to map
visParms_uhi = {
    'min': -18, 'max': 18,
    'palette': ['00008F', '0010FF', '0060FF', '00FFFF', 'CCFFCC', 'FFFF00', 'FF8000', 'FF2000', '800000']
}

Map = geemap.Map()
Map.addLayer(GetUhiStatsByYear(2020).select('UHI_Day_1km_p99'), visParms_uhi, 'UHI 2020')
Map.center_object(fc_Shanghai.geometry())
Map

Map(center=[20, 0], controls=(WidgetControl(options=['position', 'transparent_bg'], widget=HBox(children=(Togg…

## Load WorldPop

In [15]:
# Function to get WorldPop by year
def GetWorldPopByYear(year):
  ImColPop = ee.ImageCollection("WorldPop/GP/100m/pop")
  ImColPopYear = ImColPop.filter(ee.Filter.calendarRange(year,year,'year'))
  return ImColPopYear.mosaic()

# Function to get WorldPop (1km) by year
def GetWorldPop1kmByYear(year):
  ImColPop = ee.ImageCollection("WorldPop/GP/100m/pop")
  uhiModis1 = ee.ImageCollection('users/lorenzomentaschi/uhiModis');
  uhiProjection = uhiModis1.first().projection().getInfo()['crs']
  ImColPopYear = ImColPop.filter(ee.Filter.calendarRange(year,year,'year'))
  return ImColPopYear.mosaic().reduceResolution(ee.Reducer.sum()).reproject({'crs': uhiProjection, 'scale': 1000})

# Add test layer to map
visParms_pop = {
  'bands': ['population'],
  'min': 0.0,
  'max': 50.0,
  'palette': ['24126c', '1fff4f', 'd4ff50']
}

Map = geemap.Map()
Map.addLayer(GetWorldPopByYear(2020), visParms_pop, 'Population 2020')
# Map.addLayer(GetWorldPop1kmByYear(2020), visParms_pop, 'Population 2020 (1km)')
Map.center_object(fc_Shanghai.geometry())
Map
# ee.ImageCollection("WorldPop/GP/100m/pop").first().projection().getInfo()
# ee.ImageCollection('users/lorenzomentaschi/uhiModis').first().projection().getInfo()

Map(center=[20, 0], controls=(WidgetControl(options=['position', 'transparent_bg'], widget=HBox(children=(Togg…

# UHI times POP & Urban

In [11]:
# Function to get UHI times POP
def GetUhiStatsTimesPopByYear(year):
  ImUhiStat = GetUhiStatsByYear(year)
  ImPop = GetWorldPopByYear(year).updateMask(ImUhiStat.select(1))
  return ImUhiStat.multiply(ImPop)

# Function to get UHI masked by urban land
def GetUhiStatsUrbanByYear(year):
  ImUhiStat = GetUhiStatsByYear(year)
  ImUrban = GetUrbanLandByYear(year)
  return ImUhiStat.updateMask(ImUrban)

# Add test layer to map
visParms_uhipop = {
    'max': 5000,
    'palette': ['green', 'yellow', 'orange', 'red', 'purple']
}

Map = geemap.Map()
# Map.addLayer(GetUhiStatsTimesPopByYear(2020).select('UHI_Day_1km_p99'), visParms_uhipop, 'UHI x POP 2020')

Map.addLayer(GetUhiStatsUrbanByYear(2020).select('UHI_Day_1km_p99'), visParms_uhi, 'UHI 2020')
Map.center_object(fc_Shanghai.geometry())
Map

Map(center=[20, 0], controls=(WidgetControl(options=['position', 'transparent_bg'], widget=HBox(children=(Togg…

# Mean UHI, POP for FUA

## Define function to calculate UHI per cap

In [12]:
# Testing
# gdf_fua = gdf_Shanghai
# gdf_fua = gdf_Helsinki
# year = 2020

# Define function to calculate UHI per cap for FUA, in a year
def GetUhiPercapFUAbyYear(gdf_fua, year):
  fc_fua = geemap.geopandas_to_ee(gdf_fua).first()
  geo_fua = fc_fua.geometry()
  uhi_cnt = GetUhiStatsByYear(year).reduceRegion(ee.Reducer.count(), geo_fua, 1000).getInfo()
  if uhi_cnt['UHI_Day_1km_p99'] > 0:
    uhi_mean        = GetUhiStatsByYear(year).reduceRegion(ee.Reducer.mean(), geo_fua, 1000).getInfo()
    # pop_sum_100m    = GetWorldPopByYear(year).reduceRegion(ee.Reducer.sum(), geo_fua, 100).getInfo()
    pop_sum_1km     = GetWorldPopByYear(year).reduceRegion(ee.Reducer.sum(), geo_fua, 1000).getInfo()
    uhiTimesPop_sum = GetUhiStatsTimesPopByYear(year).reduceRegion(ee.Reducer.sum(), geo_fua, 1000).getInfo()
    fua_id   = gdf_fua[['eFUA_ID']].iloc[0].iloc[0]
    fua_name = gdf_fua[['eFUA_name']].iloc[0].iloc[0]
    uhi_day_p99_percap = uhiTimesPop_sum['UHI_Day_1km_p99']   / pop_sum_1km['population']
    uhi_nig_p99_percap = uhiTimesPop_sum['UHI_Night_1km_p99'] / pop_sum_1km['population']
    df_UhiPercap = pd.DataFrame({
        'Year': year,
        # 'population': pop_sum_100m['population'],
        'UHI_Day_1km_p99': [uhi_mean['UHI_Day_1km_p99']],
        'UHI_Night_1km_99': [uhi_mean['UHI_Night_1km_p99']],
        'UHI_Day_1km_p95_percap': [uhi_day_p99_percap], 
        'UHI_Night_1km_p99_percap': [uhi_nig_p99_percap],
        'eFUA_name':[fua_name], 
        'eFUA_ID':[fua_id]}, index=[fua_id])
  else:
    df_UhiPercap = pd.DataFrame({})
  return df_UhiPercap

print(GetUhiPercapFUAbyYear(gdf_Helsinki, 2020).empty)
print(GetUhiPercapFUAbyYear(gdf_Shanghai, 2020))

True
         Year  UHI_Day_1km_p99  UHI_Night_1km_99  UHI_Day_1km_p95_percap  \
10097.0  2020         5.895528          2.350958                8.002348   

         UHI_Night_1km_p99_percap eFUA_name  eFUA_ID  
10097.0                  3.587992  Shanghai  10097.0  


## Apply function to all FUAs

## First run

In [55]:
path_out = '/content/drive/MyDrive/Research_Projects/2022_UHI_vs_GHG/results/pop_weighted_UHIs/'
filename_out = path_out + 'pop_weighted_uhi_fuas.csv'

# Create a empty data frame for results
df_UhiPercap = pd.DataFrame()

# Loop through FUAs
for index, row in tqdm(gdf_FUAs.head(100).iterrows()):
  for year in range(2003, 2021):
    fua_id = row['eFUA_ID']
    gdf_fua = gdf_FUAs.loc[gdf_FUAs['eFUA_ID']==fua_id]
    df_UhiPercap_new = GetUhiPercapFUAbyYear(gdf_fua, year)
    if df_UhiPercap_new.empty==False:
      df_UhiPercap = df_UhiPercap.append(df_UhiPercap_new)
      df_UhiPercap.to_csv(filename_out)

df_UhiPercap.head(10)

100it [1:44:12, 62.52s/it]


,Year,UHI_Day_1km_p99,UHI_Night_1km_99,UHI_Day_1km_p95_percap,UHI_Night_1km_p99_percap,eFUA_name,eFUA_ID
5129.0,2003,9.866832,2.628742,12.147860,3.307194,Tokyo,5129.0
5129.0,2004,10.258894,3.101786,12.147617,4.046105,Tokyo,5129.0
5129.0,2005,10.119174,2.776083,11.918405,3.658930,Tokyo,5129.0
5129.0,2006,9.914180,2.974225,12.066969,3.876837,Tokyo,5129.0
5129.0,2007,10.675253,2.906646,12.889787,3.714579,Tokyo,5129.0
5129.0,2008,10.134808,2.546656,11.925014,3.577544,Tokyo,5129.0
5129.0,2009,10.596373,2.632958,12.771232,3.372628,Tokyo,5129.0
5129.0,2010,10.310990,3.092582,12.698195,3.916307,Tokyo,5129.0
5129.0,2011,10.741403,2.882687,12.490841,3.754795,Tokyo,5129.0
5129.0,2012,10.479428,3.134394,13.191554,4.223880,Tokyo,5129.0


## Continue previous runs

In [ ]:
path_out = '/content/drive/MyDrive/Research_Projects/2022_UHI_vs_GHG/results/pop_weighted_UHIs/'
filename_out = path_out + 'pop_weighted_uhi_fuas.csv'

# Read previous run
df_UhiPercap = pd.read_csv(filename_out)

# Loop through FUAs
for index, row in tqdm(gdf_FUAs.head(1000).iterrows()):
  for year in range(2003, 2021):
    fua_id = row['eFUA_ID']
    is_FuaID_exists = (df_UhiPercap['eFUA_ID']==fua_id) # Has the FUA been run?
    is_Year_exists = (df_UhiPercap['Year']==year)       # Has the year been run?
    # Only run the (FUA, year) combo that has not been run
    if df_UhiPercap.loc[is_FuaID_exists & is_Year_exists].empty:
      gdf_fua = gdf_FUAs.loc[gdf_FUAs['eFUA_ID']==fua_id]
      df_UhiPercap_new = GetUhiPercapFUAbyYear(gdf_fua, year)
      if df_UhiPercap_new.empty==False:
        df_UhiPercap = df_UhiPercap.append(df_UhiPercap_new)
        df_UhiPercap.to_csv(filename_out)

225it [1:33:21, 48.79s/it]